## THR->ALA vacuum from REST2 cache

### Define imports and global variables

In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm

In [2]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/11/18/"
phase = 'vacuum'
sim_number = 1
old_name = "THR"
new_name = "ALA"
endstate = 0
length = 5

In [4]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}


In [5]:
# Define simulation parameters
nsteps_eq = 1 
nsteps_neq = 250000 # 1 ns
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'

### Generate in vanilla htf

In [12]:
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [13]:
# Generate htf for capped THR->ALA in vacuum
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs=None,
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}, 
                                   molecules=None, 
                                   cache=None)

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(pdb.topology)

htf = generate_dipeptide_top_pos_sys(pdb.topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

out_dir = '/data/chodera/zhangi/perses_benchmark/neq/11/18/'
pickle.dump(htf, open(os.path.join(out_dir, "18_vacuum.pickle"), "wb" ))

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from THR to ALA


INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 98 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.9799661941737812, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (6, 7, 10, 13, [1, Quantity(value=2.105358864604291, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 4]), 2: (6, 7, 10, 14, [1, Quan

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.437656936855454
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144895211145922e-06
INFO:geometry:Final logp_proposal: -28536.38220227044


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanome

In [14]:
# # Read in vanilla htf
# with open(os.path.join(otherdir, f"{12}_{phase}.pickle"), 'rb') as f:
#     htf = pickle.load(f)

In [15]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE1-C
1 ACE1-O
2 ACE1-CH3
3 ACE1-H1
4 ACE1-H2
5 ACE1-H3
6 THR2-N
7 THR2-CA
8 THR2-C
9 THR2-O
10 THR2-CB
11 THR2-CG2
12 THR2-OG1
13 THR2-H
14 THR2-HA
15 THR2-HB
16 THR2-HG1
17 THR2-HG21
18 THR2-HG22
19 THR2-HG23
26 THR2-CB
27 THR2-HB1
28 THR2-HB2
29 THR2-HB3
20 NME3-N
21 NME3-C
22 NME3-H
23 NME3-H1
24 NME3-H2
25 NME3-H3


### Get decorrelated hybrid positions

In [16]:
# Read in nc file
from perses.analysis.utils import open_netcdf
i = os.path.basename(os.path.dirname(outdir))
nc = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns.nc"))
nc_checkpoint = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns_checkpoint.nc"))
checkpoint_interval = nc_checkpoint.CheckpointInterval
all_positions = nc_checkpoint.variables['positions']
n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)



In [17]:
# Get old, new, and hybrid positions
all_pos = np.zeros(shape=(n_iter, htf.hybrid_topology.n_atoms, 3))
all_pos_new = np.zeros(shape=(n_iter, htf._topology_proposal.new_topology.getNumAtoms(), 3))
all_pos_old = np.zeros(shape=(n_iter, htf._topology_proposal.old_topology.getNumAtoms(), 3))
for iteration in tqdm(range(n_iter)):
    replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
    pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
    all_pos[iteration] = pos
    all_pos_new[iteration] = htf.new_positions(pos).value_in_unit_system(unit.md_unit_system)
    all_pos_old[iteration] = htf.old_positions(pos).value_in_unit_system(unit.md_unit_system)

100%|██████████| 5001/5001 [00:41<00:00, 119.20it/s]


In [18]:
# Compute dihedrals of old and new positions
dihedrals_all = []
old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
dihedral_indices_new = [13, 10, 7, 6]
dihedral_indices_old = [12, 10, 7, 6]
for pos, top, indices in zip([all_pos_new, all_pos_old], [new_top, old_top], [dihedral_indices_new, dihedral_indices_old]):
    traj = md.Trajectory(np.array(pos), top)
    dihedrals = md.compute_dihedrals(traj, np.array([indices]))
    dihedrals_all.append(dihedrals)

In [19]:
# Get decorrelated indices for THR dihedral
from perses.dispersed import feptasks
t0, g, neff_max, a_t, uncorrelated_indices = feptasks.compute_timeseries(dihedrals_all[1]) # dihedrals of old atoms

In [20]:
uncorrelated_indices

[2677,
 2681,
 2685,
 2689,
 2692,
 2696,
 2700,
 2704,
 2708,
 2712,
 2716,
 2719,
 2723,
 2727,
 2731,
 2735,
 2739,
 2743,
 2746,
 2750,
 2754,
 2758,
 2762,
 2766,
 2770,
 2774,
 2777,
 2781,
 2785,
 2789,
 2793,
 2797,
 2801,
 2804,
 2808,
 2812,
 2816,
 2820,
 2824,
 2828,
 2831,
 2835,
 2839,
 2843,
 2847,
 2851,
 2855,
 2858,
 2862,
 2866,
 2870,
 2874,
 2878,
 2882,
 2885,
 2889,
 2893,
 2897,
 2901,
 2905,
 2909,
 2913,
 2916,
 2920,
 2924,
 2928,
 2932,
 2936,
 2940,
 2943,
 2947,
 2951,
 2955,
 2959,
 2963,
 2967,
 2970,
 2974,
 2978,
 2982,
 2986,
 2990,
 2994,
 2997,
 3001,
 3005,
 3009,
 3013,
 3017,
 3021,
 3024,
 3028,
 3032,
 3036,
 3040,
 3044,
 3048,
 3051,
 3055,
 3059,
 3063,
 3067,
 3071,
 3075,
 3079,
 3082,
 3086,
 3090,
 3094,
 3098,
 3102,
 3106,
 3109,
 3113,
 3117,
 3121,
 3125,
 3129,
 3133,
 3136,
 3140,
 3144,
 3148,
 3152,
 3156,
 3160,
 3163,
 3167,
 3171,
 3175,
 3179,
 3183,
 3187,
 3190,
 3194,
 3198,
 3202,
 3206,
 3210,
 3214,
 3217,
 3221,
 3225,

In [21]:
len(uncorrelated_indices)

602

In [22]:
# Choose 100 random indices from uncorrelated indices
import random
subset_indices = random.choices(uncorrelated_indices, k=100)

In [23]:
# Make array of hybrid positions for 100 uncorrelated indices
subset_pos = all_pos[subset_indices]

In [24]:
with open(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns_snapshots.npy"), 'wb') as f:
    np.save(f, subset_pos)

In [ ]:
# # Read in cache
# with open(os.path.join(args.dir, f"{i}_{phase}_{old_name.lower()}_{length}ns_snapshots.npy"), 'rb') as f:
#     pos_hybrid = np.load(f)

In [25]:
# Set system and positions 
system = htf.hybrid_system
positions = subset_pos[sim_number-1]


In [26]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)



In [27]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)

In [28]:
# Run eq forward (0 -> 1)
initial_time = time.time()
integrator.step(nsteps_eq)
elapsed_time = (time.time() - initial_time) * unit.seconds
_logger.info(f'Equilibrating at lambda = 0, took: {elapsed_time / unit.seconds} seconds')

# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
for fwd_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    forward_works.append(integrator.get_protocol_work(dimensionless=True))
    if fwd_step % 2500 == 0:
        _logger.info(f'forward NEQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        old_pos = np.asarray(htf.old_positions(pos))
        new_pos = np.asarray(htf.new_positions(pos))
        forward_neq_old.append(old_pos)
        forward_neq_new.append(new_pos)
forward_works_master.append(forward_works)



INFO:root:Equilibrating at lambda = 0, took: 1.0948553085327148 seconds
INFO:root:forward NEQ step: 0, took: 0.00047087669372558594 seconds
INFO:root:forward NEQ step: 2500, took: 0.0003371238708496094 seconds
INFO:root:forward NEQ step: 5000, took: 0.00036597251892089844 seconds
INFO:root:forward NEQ step: 7500, took: 0.0003628730773925781 seconds
INFO:root:forward NEQ step: 10000, took: 0.0003657341003417969 seconds
INFO:root:forward NEQ step: 12500, took: 0.0003333091735839844 seconds
INFO:root:forward NEQ step: 15000, took: 0.00037479400634765625 seconds
INFO:root:forward NEQ step: 17500, took: 0.00035858154296875 seconds
INFO:root:forward NEQ step: 20000, took: 0.00035309791564941406 seconds
INFO:root:forward NEQ step: 22500, took: 0.00033020973205566406 seconds
INFO:root:forward NEQ step: 25000, took: 0.0003600120544433594 seconds
INFO:root:forward NEQ step: 27500, took: 0.00037980079650878906 seconds
INFO:root:forward NEQ step: 30000, took: 0.0003533363342285156 seconds
INFO:roo

KeyboardInterrupt: 

In [ ]:
# Load cache for ALA endstate hybrid positions

In [ ]:
# Run eq reverse (1 -> 0)
initial_time = time.time()
integrator.step(nsteps_eq)
elapsed_time = (time.time() - initial_time) * unit.seconds
_logger.info(f'Equilibrating at lambda = 1, took: {elapsed_time / unit.seconds} seconds')


In [ ]:
# Run neq reverse (1 -> 0)
reverse_works_master = list()
reverse_neq_old, reverse_neq_new = list(), list()
reverse_works = [integrator.get_protocol_work(dimensionless=True)]
for rev_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    reverse_works.append(integrator.get_protocol_work(dimensionless=True))
    if rev_step % 2500 == 0:
        _logger.info(f'reverse NEQ step: {rev_step}, took: {elapsed_time / unit.seconds} seconds')
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        old_pos = np.asarray(htf.old_positions(pos))
        new_pos = np.asarray(htf.new_positions(pos))
        reverse_neq_old.append(old_pos)
        reverse_neq_new.append(new_pos)
reverse_works_master.append(reverse_works)


# Save works
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward.npy"), 'wb') as f:
    np.save(f, forward_works_master)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse.npy"), 'wb') as f:
    np.save(f, reverse_works_master)

# Save trajs
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_neq_old.npy"), 'wb') as f:
    np.save(f, forward_neq_old)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_neq_new.npy"), 'wb') as f:
    np.save(f, forward_neq_new)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_neq_old.npy"), 'wb') as f:
    np.save(f, reverse_neq_old)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_neq_new.npy"), 'wb') as f:
    np.save(f, reverse_neq_new)


## THR->ALA solvent from REST2 cache

In [3]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm

In [4]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/11/18/"
i = os.path.basename(os.path.dirname(outdir))
phase = 'solvent'
sim_number = 1
old_name = "THR"
new_name = "ALA"
endstate = 0
length = 5

In [9]:
# Read in vanilla htf
with open(os.path.join(outdir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)

### Get decorrelated hybrid positions

In [10]:
# Read in nc file
from perses.analysis.utils import open_netcdf
nc = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns.nc"))
nc_checkpoint = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns_checkpoint.nc"))
checkpoint_interval = nc_checkpoint.CheckpointInterval
all_positions = nc_checkpoint.variables['positions']
n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)



In [11]:
# Get old, new, and hybrid positions
all_pos = np.zeros(shape=(n_iter, htf.hybrid_topology.n_atoms, 3))
all_pos_new = np.zeros(shape=(n_iter, htf._topology_proposal.new_topology.getNumAtoms(), 3))
all_pos_old = np.zeros(shape=(n_iter, htf._topology_proposal.old_topology.getNumAtoms(), 3))
for iteration in tqdm(range(n_iter)):
    replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
    pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
    all_pos[iteration] = pos
    all_pos_new[iteration] = htf.new_positions(pos).value_in_unit_system(unit.md_unit_system)
    all_pos_old[iteration] = htf.old_positions(pos).value_in_unit_system(unit.md_unit_system)

100%|██████████| 5001/5001 [27:11<00:00,  3.07it/s] 


In [12]:
# Compute dihedrals of old and new positions
dihedrals_all = []
old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
dihedral_indices_new = [13, 10, 7, 6]
dihedral_indices_old = [12, 10, 7, 6]
for pos, top, indices in zip([all_pos_new, all_pos_old], [new_top, old_top], [dihedral_indices_new, dihedral_indices_old]):
    traj = md.Trajectory(np.array(pos), top)
    dihedrals = md.compute_dihedrals(traj, np.array([indices]))
    dihedrals_all.append(dihedrals)

In [ ]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom)

In [ ]:
for atom in htf._topology_proposal.old_topology.atoms():
    print(atom)

In [19]:
# Get decorrelated indices for THR dihedral
from perses.dispersed import feptasks
t0, g, neff_max, a_t, uncorrelated_indices = feptasks.compute_timeseries(dihedrals_all[1]) # dihedrals of old atoms

In [20]:
uncorrelated_indices

[2677,
 2681,
 2685,
 2689,
 2692,
 2696,
 2700,
 2704,
 2708,
 2712,
 2716,
 2719,
 2723,
 2727,
 2731,
 2735,
 2739,
 2743,
 2746,
 2750,
 2754,
 2758,
 2762,
 2766,
 2770,
 2774,
 2777,
 2781,
 2785,
 2789,
 2793,
 2797,
 2801,
 2804,
 2808,
 2812,
 2816,
 2820,
 2824,
 2828,
 2831,
 2835,
 2839,
 2843,
 2847,
 2851,
 2855,
 2858,
 2862,
 2866,
 2870,
 2874,
 2878,
 2882,
 2885,
 2889,
 2893,
 2897,
 2901,
 2905,
 2909,
 2913,
 2916,
 2920,
 2924,
 2928,
 2932,
 2936,
 2940,
 2943,
 2947,
 2951,
 2955,
 2959,
 2963,
 2967,
 2970,
 2974,
 2978,
 2982,
 2986,
 2990,
 2994,
 2997,
 3001,
 3005,
 3009,
 3013,
 3017,
 3021,
 3024,
 3028,
 3032,
 3036,
 3040,
 3044,
 3048,
 3051,
 3055,
 3059,
 3063,
 3067,
 3071,
 3075,
 3079,
 3082,
 3086,
 3090,
 3094,
 3098,
 3102,
 3106,
 3109,
 3113,
 3117,
 3121,
 3125,
 3129,
 3133,
 3136,
 3140,
 3144,
 3148,
 3152,
 3156,
 3160,
 3163,
 3167,
 3171,
 3175,
 3179,
 3183,
 3187,
 3190,
 3194,
 3198,
 3202,
 3206,
 3210,
 3214,
 3217,
 3221,
 3225,

In [21]:
len(uncorrelated_indices)

602

In [22]:
# Choose 100 random indices from uncorrelated indices
import random
subset_indices = random.choices(uncorrelated_indices, k=100)

In [23]:
# Make array of hybrid positions for 100 uncorrelated indices
subset_pos = all_pos[subset_indices]

In [24]:
with open(os.path.join(outdir, f"{i}_{phase}_{old_name.lower()}_{length}ns_snapshots.npy"), 'wb') as f:
    np.save(f, subset_pos)